<a href="https://colab.research.google.com/github/puru444/Neuron_X/blob/main/content_type_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [2]:
# Upload the "content_type.xlxs" file into Colab, then store in a Pandas DataFrame

from google.colab import files
uploaded = files.upload()

df_content_type = pd.read_excel(
    ("content_type.xlsx"))

display(df_content_type.head(500))
display(df_content_type.tail(500))

Saving content_type.xlsx to content_type.xlsx


,Gender,Subscription_Plan,Age_Group,Renewal_Date_within_Days,Content_Division,Time_Slot,Time_Spent_in_Min,Categories,Content_Type
0,Male,Regular,16-20,7,Hollywood,06AM-10AM,1 to 15,Comedy,0.0
1,Female,Silver,20-30,15,Latino,10AM-02PM,15 to 30,Action,1.0
2,Male,Gold,30-40,30,Bollywood,02PM-06PM,30 to 60,SciFi,1.0
3,Male,Regular,40-50,45,Mandarin,06PM-10PM,60 to 120,Horror,1.0
4,Male,Silver,50-60,60,Arabic,10PM-02AM,120-180,Adventure,0.0
...,...,...,...,...,...,...,...,...,...
495,Female,Regular,50-60,30,Others,06AM-10AM,15 to 30,Romance,1.0
496,Male,Regular,40-50,45,Others,10AM-02PM,120-180,Animated,0.0
497,Female,Silver,16-20,7,Bollywood,06PM-10PM,60 to 120,Musical,1.0
498,Male,Silver,30-40,15,Hollywood,10AM-02PM,120-180,Thriller,0.0


,Gender,Subscription_Plan,Age_Group,Renewal_Date_within_Days,Content_Division,Time_Slot,Time_Spent_in_Min,Categories,Content_Type
538,Male,Regular,16-20,30,Others,06PM-10PM,30 to 60,Action,1.0
539,Female,Silver,50-60,45,Others,06AM-10AM,1 to 15,Musical,0.0
540,Female,Silver,50-60,7,Bollywood,02PM-06PM,30 to 60,Romance,1.0
541,Male,Silver,20-30,15,Hollywood,10PM-02AM,15 to 30,Fantasy,0.0
542,Male,Gold,30-40,7,Others,10AM-02PM,30 to 60,Comedy,0.0
...,...,...,...,...,...,...,...,...,...
1033,Female,Regular,40-50,45,Hollywood,06AM-10AM,120-180,Thriller,0.0
1034,Male,Silver,20-30,7,Mandarin,10PM-02AM,60 to 120,Comedy,0.0
1035,Female,Silver,16-20,7,Latino,10AM-02PM,60 to 120,SciFi,1.0
1036,Female,Silver,30-40,60,Bollywood,02AM-06AM,1 to 15,SciFi,1.0


In [9]:
df_content_type = df_content_type.dropna()

In [10]:

# Review the data types associated with the columns
df_content_type.dtypes

Gender                       object
Subscription_Plan            object
Age_Group                    object
Renewal_Date_within_Days      int64
Content_Division             object
Time_Slot                    object
Time_Spent_in_Min            object
Categories                   object
Content_Type                float64
dtype: object

In [11]:
# Create a list of categorical variables 
categorical_variables = list(df_content_type.dtypes[df_content_type.dtypes == "object"].index)

# Display the categorical variables list
categorical_variables

['Gender',
 'Subscription_Plan',
 'Age_Group',
 'Content_Division',
 'Time_Slot',
 'Time_Spent_in_Min',
 'Categories']

In [12]:
#np.any(np.isnan(mat))

In [13]:
#np.all(np.isfinite(mat))

In [14]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [15]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(df_content_type[categorical_variables])

In [16]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
)

# Display the DataFrame
display(encoded_df.head())

,Gender_Female,Gender_Male,Subscription_Plan_Gold,Subscription_Plan_Gole,Subscription_Plan_Regular,Subscription_Plan_Regular,Subscription_Plan_Silver,Age_Group_16-20,Age_Group_20-30,Age_Group_30-40,Age_Group_40-50,Age_Group_50-60,Age_Group_60+,Content_Division_Arabic,Content_Division_Bollywood,Content_Division_Hollywood,Content_Division_Latino,Content_Division_Mandarin,Content_Division_Others,Time_Slot_02AM-06AM,Time_Slot_02PM-06PM,Time_Slot_06AM-10AM,Time_Slot_06PM-10PM,Time_Slot_10AM-02PM,Time_Slot_10PM-02AM,Time_Spent_in_Min_1 to 15,Time_Spent_in_Min_120-180,Time_Spent_in_Min_15 to 30,Time_Spent_in_Min_180+,Time_Spent_in_Min_30 to 60,Time_Spent_in_Min_60 to 120,Categories_Action,Categories_Adventure,Categories_Animated,Categories_Comedy,Categories_Fantasy,Categories_Horror,Categories_Musical,Categories_Romance,Categories_SciFi,Categories_Thriller
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df = pd.concat([
    df_content_type.drop(columns=categorical_variables),
    encoded_df,
], axis=1)

# Review the Dataframe
encoded_df.head()

,Renewal_Date_within_Days,Content_Type,Gender_Female,Gender_Male,Subscription_Plan_Gold,Subscription_Plan_Gole,Subscription_Plan_Regular,Subscription_Plan_Regular,Subscription_Plan_Silver,Age_Group_16-20,Age_Group_20-30,Age_Group_30-40,Age_Group_40-50,Age_Group_50-60,Age_Group_60+,Content_Division_Arabic,Content_Division_Bollywood,Content_Division_Hollywood,Content_Division_Latino,Content_Division_Mandarin,Content_Division_Others,Time_Slot_02AM-06AM,Time_Slot_02PM-06PM,Time_Slot_06AM-10AM,Time_Slot_06PM-10PM,Time_Slot_10AM-02PM,Time_Slot_10PM-02AM,Time_Spent_in_Min_1 to 15,Time_Spent_in_Min_120-180,Time_Spent_in_Min_15 to 30,Time_Spent_in_Min_180+,Time_Spent_in_Min_30 to 60,Time_Spent_in_Min_60 to 120,Categories_Action,Categories_Adventure,Categories_Animated,Categories_Comedy,Categories_Fantasy,Categories_Horror,Categories_Musical,Categories_Romance,Categories_SciFi,Categories_Thriller
0,7.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,45.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,60.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df["Content_Type"]

# Display a sample of y
y[:5]

0    0.0
1    1.0
2    1.0
3    1.0
4    0.0
Name: Content_Type, dtype: float64

In [19]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns=["Content_Type"])

# Review the features DataFrame
display(X.head())
display(X.tail())

,Renewal_Date_within_Days,Gender_Female,Gender_Male,Subscription_Plan_Gold,Subscription_Plan_Gole,Subscription_Plan_Regular,Subscription_Plan_Regular,Subscription_Plan_Silver,Age_Group_16-20,Age_Group_20-30,Age_Group_30-40,Age_Group_40-50,Age_Group_50-60,Age_Group_60+,Content_Division_Arabic,Content_Division_Bollywood,Content_Division_Hollywood,Content_Division_Latino,Content_Division_Mandarin,Content_Division_Others,Time_Slot_02AM-06AM,Time_Slot_02PM-06PM,Time_Slot_06AM-10AM,Time_Slot_06PM-10PM,Time_Slot_10AM-02PM,Time_Slot_10PM-02AM,Time_Spent_in_Min_1 to 15,Time_Spent_in_Min_120-180,Time_Spent_in_Min_15 to 30,Time_Spent_in_Min_180+,Time_Spent_in_Min_30 to 60,Time_Spent_in_Min_60 to 120,Categories_Action,Categories_Adventure,Categories_Animated,Categories_Comedy,Categories_Fantasy,Categories_Horror,Categories_Musical,Categories_Romance,Categories_SciFi,Categories_Thriller
0,7.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,45.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,60.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Renewal_Date_within_Days,Gender_Female,Gender_Male,Subscription_Plan_Gold,Subscription_Plan_Gole,Subscription_Plan_Regular,Subscription_Plan_Regular,Subscription_Plan_Silver,Age_Group_16-20,Age_Group_20-30,Age_Group_30-40,Age_Group_40-50,Age_Group_50-60,Age_Group_60+,Content_Division_Arabic,Content_Division_Bollywood,Content_Division_Hollywood,Content_Division_Latino,Content_Division_Mandarin,Content_Division_Others,Time_Slot_02AM-06AM,Time_Slot_02PM-06PM,Time_Slot_06AM-10AM,Time_Slot_06PM-10PM,Time_Slot_10AM-02PM,Time_Slot_10PM-02AM,Time_Spent_in_Min_1 to 15,Time_Spent_in_Min_120-180,Time_Spent_in_Min_15 to 30,Time_Spent_in_Min_180+,Time_Spent_in_Min_30 to 60,Time_Spent_in_Min_60 to 120,Categories_Action,Categories_Adventure,Categories_Animated,Categories_Comedy,Categories_Fantasy,Categories_Horror,Categories_Musical,Categories_Romance,Categories_SciFi,Categories_Thriller
1033,45.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1034,7.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1035,7.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1036,60.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1037,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:

# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [21]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# Define the the number of inputs (features) to the model
number_input_features = X_train_scaled.shape[1]

# Review the number of features
number_input_features

42

In [24]:
# Define the number of neurons in the output layer
number_output_neurons = 1

# Review the number of neurons in the output layer
number_output_neurons

1

In [25]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  (number_input_features + number_output_neurons) // 2

# Review the number hidden nodes in the first layer
hidden_nodes_layer1

21

In [26]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  (hidden_nodes_layer1 + number_output_neurons) // 2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2

11

In [28]:

# Create the Sequential model instance
nn = Sequential()


# Add the first hidden layer
nn.add(Dense(
    units=hidden_nodes_layer1, 
    input_dim=number_input_features, 
    activation="relu")
)


# Add the second hidden layer
nn.add(Dense(
    units=hidden_nodes_layer2, 
    activation="relu")
)


# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(
    1, 
    activation="sigmoid")
)

In [29]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 21)                903       
_________________________________________________________________
dense_4 (Dense)              (None, 11)                242       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 12        
Total params: 1,157
Trainable params: 1,157
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
# Fit the model using 50 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
25/25 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.5386
Epoch 2/50
25/25 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5386
Epoch 3/50
25/25 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5386
Epoch 4/50
25/25 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5386
Epoch 5/50
25/25 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5386
Epoch 6/50
25/25 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5386
Epoch 7/50
25/25 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5386
Epoch 8/50
25/25 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5386
Epoch 9/50
25/25 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5386
Epoch 10/50
25/25 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5386
Epoch 11/50
25/25 [==============================

In [36]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

9/9 - 0s - loss: nan - accuracy: 0.5115
Loss: nan, Accuracy: 0.5115384459495544


In [37]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

42

In [38]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 1


In [39]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 = (number_input_features + number_output_neurons_A1) // 2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A1

21